In [7]:
import os
import numpy as np
import cv2
from scipy.stats import skew, kurtosis
from sklearn.preprocessing import StandardScaler
from collections import Counter


In [8]:
def extract_statistical_features(image_vector):
    features = []

    skew_value = skew(image_vector)
    skew_value = 0 if np.isnan(skew_value) else skew_value
    features.append(skew_value)

    kurtosis_value = kurtosis(image_vector)
    kurtosis_value = 0 if np.isnan(kurtosis_value) else kurtosis_value
    features.append(kurtosis_value)

    rms_value = np.sqrt(np.mean(np.square(image_vector)))
    rms_value = 1 if rms_value == 0 else rms_value
    crest_factor = np.max(image_vector) / rms_value
    features.append(crest_factor)

    mean_absolute_value = np.mean(np.abs(image_vector))
    mean_absolute_value = 1 if mean_absolute_value == 0 else mean_absolute_value
    shape_factor = rms_value / mean_absolute_value
    features.append(shape_factor)

    impulse_factor = np.max(image_vector) / mean_absolute_value
    features.append(impulse_factor)

    variance_value = np.var(image_vector)
    variance_value = 1 if variance_value == 0 else variance_value
    margin_factor = np.max(image_vector) / variance_value
    features.append(margin_factor)

    return features

In [9]:
def load_images_and_extract_features(folder):
    images = []
    labels = []
    for label in os.listdir(folder):
        label_folder = os.path.join(folder, label)
        if not os.path.isdir(label_folder):
            continue

        for filename in os.listdir(label_folder):
            img_path = os.path.join(label_folder, filename)
            valid_extensions = ('.jpg', '.jpeg')
            if not filename.lower().endswith(valid_extensions):
                continue

            img = cv2.imread(img_path)
            if img is not None:
                try:
                    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                    resized = cv2.resize(gray, (128, 128))

                    sobelx = cv2.Sobel(resized, cv2.CV_16S, 1, 0, ksize=3)
                    sobely = cv2.Sobel(resized, cv2.CV_16S, 0, 1, ksize=3)

                    abs_sobelx = cv2.convertScaleAbs(sobelx)
                    abs_sobely = cv2.convertScaleAbs(sobely)
                    gradient = cv2.addWeighted(abs_sobelx, 0.5, abs_sobely, 0.5, 0)

                    gradient_vector = gradient.flatten().astype(np.float32)

                    features = extract_statistical_features(gradient_vector)

                    images.append(features)
                    labels.append(label.lower())

                except Exception as e:
                    print(f"Error processing {img_path}: {e}")
                    continue

    return np.array(images, dtype=np.float32), np.array(labels)

In [10]:
def KNN(x_train, y_train, sample_test, k):

    distances = np.linalg.norm(x_train - sample_test, axis=1)

    neighbor_indices = np.argsort(distances)[:k]

    neighbor_labels = y_train[neighbor_indices]

    most_common = Counter(neighbor_labels).most_common(1)[0][0]

    return most_common


In [11]:
train_folder = '/content/train'
test_folder = '/content/test'

print("Eğitim verileri yükleniyor...")
x_train, y_train = load_images_and_extract_features(train_folder)
print(f"x_train shape: {x_train.shape}")
print(f"y_train shape: {y_train.shape}")

print("\nTest verileri yükleniyor...")
x_test, y_test = load_images_and_extract_features(test_folder)
print(f"x_test shape: {x_test.shape}")
print(f"y_test shape: {y_test.shape}")

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

y_test_pred_labels = []

for i in range(len(x_test_scaled)):
    sample_test = x_test_scaled[i]
    predicted_class = KNN(x_train_scaled, y_train, sample_test, k=5)
    y_test_pred_labels.append(predicted_class)

for i in range(len(y_test_pred_labels)):
    print(f"Test Sample {i+1}: Predicted class = {y_test_pred_labels[i]}, Actual class = {y_test[i]}")

Eğitim verileri yükleniyor...
x_train shape: (135, 6)
y_train shape: (135,)

Test verileri yükleniyor...
x_test shape: (19, 6)
y_test shape: (19,)
Test Sample 1: Predicted class = camera, Actual class = camera
Test Sample 2: Predicted class = flamingo, Actual class = camera
Test Sample 3: Predicted class = camera, Actual class = camera
Test Sample 4: Predicted class = pizza, Actual class = camera
Test Sample 5: Predicted class = flamingo, Actual class = camera
Test Sample 6: Predicted class = flamingo, Actual class = flamingo
Test Sample 7: Predicted class = flamingo, Actual class = flamingo
Test Sample 8: Predicted class = flamingo, Actual class = flamingo
Test Sample 9: Predicted class = flamingo, Actual class = flamingo
Test Sample 10: Predicted class = flamingo, Actual class = flamingo
Test Sample 11: Predicted class = flamingo, Actual class = flamingo
Test Sample 12: Predicted class = camera, Actual class = flamingo
Test Sample 13: Predicted class = flamingo, Actual class = flamin